In [1]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import pandas as pd
import re
from sklearn.preprocessing import MinMaxScaler

In [13]:
veri = pd.read_excel('evilanlari.xlsx')

# "Oda Sayısı" sütunundaki "+" ve "1" karakterlerini temizle
veri['Oda Sayısı'] = veri['Oda Sayısı'].str.replace('+', '').str.replace('1', '').str.strip()

# "Metrekare" sütunundaki "m²" ifadelerini sil
veri['Metrekare'] = veri['Metrekare'].str.replace('m²', '').str.strip()

# "Daire Katı" sütunundaki "." ve "Kat" ifadelerini sil
veri['Daire Katı'] = veri['Daire Katı'].str.replace('.', '').str.replace('Kat', '').str.strip()

# "Fiyat" sütunundaki noktaları ve "TL" ifadelerini sil
veri['Fiyat'] = veri['Fiyat'].str.replace('.', '').str.replace('TL', '').str.strip()

# String içindeki sayıları alarak yeni bir sütun ekleyin
veri['Bina Yaşı'] = veri['Bina Yaşı'].str.extract('(\d+)', expand=False).fillna(0).astype(int)

# String değerleri ve boş hücreleri 0 ile değiştirin
veri['Oda Sayısı'] = veri['Oda Sayısı'].replace('', 0).apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
veri['Daire Katı'] = veri['Daire Katı'].replace('', 0).apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

veri.to_excel('evilanlari_temizlenmis.xlsx', index=False)

In [22]:
from sklearn.model_selection import train_test_split

hazırveri = pd.read_excel('evilanlari_temizlenmis.xlsx')

X = hazırveri.drop(['Fiyat'], axis=1)
y = hazırveri['Fiyat']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0 )

print('x_train :', x_train.shape)
print('x_test :', x_test.shape)
print('y_train :', y_train.shape)
print('y_test :', y_test.shape)

x_train : (645, 4)
x_test : (162, 4)
y_train : (645,)
y_test : (162,)


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Lineer Regresyon Train
lr = LinearRegression()
lr.fit(x_train,y_train)
y_predict = lr.predict(x_test) 

# Lasso Regression
lasso = Lasso()
lasso.fit(x_train, y_train)
y_predict_lasso = lasso.predict(x_test)

# Ridge Regression
ridge = Ridge()
ridge.fit(x_train, y_train)
y_predict_ridge = ridge.predict(x_test)

# ElasticNet Regression
elastic_net = ElasticNet()
elastic_net.fit(x_train, y_train)
y_predict_elastic_net = elastic_net.predict(x_test)

# Decision Tree Regressor
decision_tree = DecisionTreeRegressor()
decision_tree.fit(x_train, y_train)
y_predict_decision_tree = decision_tree.predict(x_test)

# Random Forest Regressor
random_forest = RandomForestRegressor()
random_forest.fit(x_train, y_train)
y_predict_random_forest = random_forest.predict(x_test)

# Gradient Boosting Regressor
gradient_boosting = GradientBoostingRegressor()
gradient_boosting.fit(x_train, y_train)
y_predict_gradient_boosting = gradient_boosting.predict(x_test)

# Performans ölçümlerini yazdırma
models = {
    'Linear Regression': y_predict,
    'Lasso Regression': y_predict_lasso,
    'Ridge Regression': y_predict_ridge,
    'ElasticNet Regression': y_predict_elastic_net,
    'Decision Tree Regressor': y_predict_decision_tree,
    'Random Forest Regressor': y_predict_random_forest,
    'Gradient Boosting Regressor': y_predict_gradient_boosting
}

for model_name, y_predict in models.items():
    print(f"{model_name}:")
    print('Mean Absolute Error:', mean_absolute_error(y_test, y_predict))
    print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_predict)))
    print('Coefficient of determination R^2:', r2_score(y_test, y_predict))
    print('-' * 40)


Linear Regression:
Mean Absolute Error: 880721.9081790159
Root Mean Squared Error: 1412677.4196972153
Coefficient of determination R^2: 0.3848936948606816
----------------------------------------
Lasso Regression:
Mean Absolute Error: 880722.1594590989
Root Mean Squared Error: 1412677.4549583439
Coefficient of determination R^2: 0.38489366415396464
----------------------------------------
Ridge Regression:
Mean Absolute Error: 880732.3823516831
Root Mean Squared Error: 1412678.9004395485
Coefficient of determination R^2: 0.3848924053739098
----------------------------------------
ElasticNet Regression:
Mean Absolute Error: 882312.2704180374
Root Mean Squared Error: 1412750.9698843644
Coefficient of determination R^2: 0.3848296429250365
----------------------------------------
Decision Tree Regressor:
Mean Absolute Error: 969697.5308641975
Root Mean Squared Error: 1733855.641668419
Coefficient of determination R^2: 0.07340480509484337
----------------------------------------
Random Fore